In [8]:
%matplotlib inline
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
from tqdm import tqdm
import pandas as pd
import ast

In [11]:
# Best stationing results
best_solutions_searched = (
    "/Users/josepaolo-t/Developer/git/mta_simulator_redo/data_analysis/data/best_solutions.parquet"
)
df = pd.read_parquet(best_solutions_searched)
df["solution"] = df["solution"].apply(ast.literal_eval)

depot = df.loc[df["date"] == "202100722"].solution.iloc[0]
type(depot)

IndexError: single positional indexer is out-of-bounds

In [30]:
numeric_bus_cols = [
    "bus_id",
    "dwell_time",
    "agg_delay",
    "service_time",
    "total_served",
    "service_kms",
    "status",
    "total_deadsecs",
]

numeric_stop_cols = ["total_passenger_ons", "total_passenger_offs", "total_passenger_walk_away"]


def logs_to_df(mypath):
    log_file_arr = {}

    bus_metrics = []
    stop_metrics = []

    start_bus_metrics = False
    start_stop_metrics = False

    log_path = f"{mypath}"
    with open(log_path) as file:
        for line in file:
            data = line.rstrip()

            if (
                "bus_id,dwell_time,agg_delay,service_time,total_served,service_kms,current_stop,status,type,total_deadsecs,starting_stop"
                in line
            ):
                bus_metrics.append(data)
                start_stop_metrics = False
                start_bus_metrics = True
                continue

            if "stop_id,total_passenger_ons,total_passenger_offs,total_passenger_walk_away" in line:
                stop_metrics.append(data)
                start_stop_metrics = True
                start_bus_metrics = False
                continue

            if "Count of all passengers" in line:
                break

            if "passenger waiting dict list" in line:
                break

            if start_bus_metrics:
                bus_metrics.append(data)

            if start_stop_metrics:
                stop_metrics.append(data)

        # break
        bus_df = pd.DataFrame([sub.split(",") for sub in bus_metrics])
        # if bus_df.empty:
        #     continue
        # display(bus_df.head(1))
        new_header = bus_df.iloc[0]  # grab the first row for the header
        bus_df = bus_df[1:]  # take the data less the header row
        bus_df.columns = new_header  # set the header row as the df header

        bus_df[numeric_bus_cols] = bus_df[numeric_bus_cols].apply(pd.to_numeric, errors="coerce")

        stop_df = pd.DataFrame([sub.split(",") for sub in stop_metrics])
        # display(stop_df.head(1))
        new_header = stop_df.iloc[0]  # grab the first row for the header
        stop_df = stop_df[1:]  # take the data less the header row
        stop_df.columns = new_header  # set the header row as the df header

        stop_df[numeric_stop_cols] = stop_df[numeric_stop_cols].apply(pd.to_numeric, errors="coerce")
        stop_df = stop_df[["stop_id", "total_passenger_ons", "total_passenger_offs", "total_passenger_walk_away"]]
    #     # stop_df
    #     label = "MCTS"
    #     if baselines:
    #         label = "Greedy"
    #     nr_label = "With Reallocation"
    #     if norealloc:
    #         nr_label = "No Reallocation"
    #     log_file_arr[(date, label, nr_label)] = [bus_df, stop_df]
    # return log_file_arr
    return stop_df, bus_df

In [31]:
res_path = (
    "/Users/josepaolo-t/Developer/git/mta_simulator_redo/code_root/experiments/TEST/results/20210211_test/results.csv"
)
sdf, bdf = logs_to_df(res_path)
bdf.head()

,bus_id,dwell_time,agg_delay,service_time,total_served,service_kms,current_stop,status,type,total_deadsecs,starting_stop
1,120,10172.1,0.00,23214.0,63.0,0.0,CXONGULC,2,BusType.REGULAR,0.0,WALMARTC
2,1203,7429.2,5.68,13485.0,55.0,0.0,MURSTOWF,2,BusType.REGULAR,0.0,MTSU
3,121,9987.8,0.99,23560.0,124.0,0.0,LAF1AVWN,2,BusType.REGULAR,0.0,HICHICNN
4,124,8881.5,0.00,19483.0,20.0,0.0,CXI17AVE,2,BusType.REGULAR,0.0,MCC
5,131,11316.1,0.00,25055.0,28.0,0.0,PREWILNN,2,BusType.REGULAR,0.0,WALMARTC


In [33]:
sdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 1 to 840
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   stop_id                    840 non-null    object 
 1   total_passenger_ons        840 non-null    float64
 2   total_passenger_offs       840 non-null    float64
 3   total_passenger_walk_away  840 non-null    float64
dtypes: float64(3), object(1)
memory usage: 26.4+ KB
